In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

In [2]:
PM10 = pd.read_csv('./data/PM10-2023.csv', sep=';')
NO2 = pd.read_csv('./data/NO2-2023.csv', sep=';')
traffic = pd.DataFrame()
for file in os.listdir('./data/natezenie_ruchu_2023'):
    data = pd.read_csv('./data/natezenie_ruchu_2023/' + file, sep=';')
    traffic = pd.concat([traffic, data])

In [3]:
PM10 = PM10.drop_duplicates()
traffic = traffic.drop_duplicates()
NO2 = NO2.drop_duplicates()

In [4]:
traffic.columns = ['station_id', 'date', 'hour', 'quality', 'num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed', 'num_trucks', 'avg_truck_speed']
df = pd.read_excel('./data/Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
# a = []
# for i, item in enumerate(df['DET_ID15']):
#     if item in traffic['station_id'].unique():
#         a.append([df['x'][i], df['y'][i]])

In [5]:
traffic_id = [100101010031590, 100101010031287]
traffic['date'] = pd.to_datetime(traffic['date'], format="%d.%m.%Y")
traffic['hour'] = pd.to_timedelta(traffic['hour'], unit='h')
traffic['datetime'] = traffic['date'] + traffic['hour']
traffic = traffic.drop(columns=['date', 'hour'])
traffic = traffic[traffic['station_id'].isin(traffic_id)]

In [6]:
PM10['Date'] = pd.to_datetime(PM10['Date'], format="%Y-%m-%d")
PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
PM10['Time'][PM10['Time'] == '24:00'] = '00:00'
PM10['Time'] = pd.to_timedelta(PM10['Time'].str.strip("'") + ':00')
PM10['datetime'] = PM10['Date'] + PM10['Time']
PM10 = PM10.drop(columns=['Date', 'Time'])

C:\Users\wpietron\AppData\Local\Temp\ipykernel_10592\4234781329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
C:\Users\wpietron\AppData\Local\Temp\ipykernel_10592\4234781329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Time'][PM10['Time'] == '24:00'] = '00:00'


In [7]:
print(len(traffic))
print(len(PM10))

13432
17518


In [8]:
#date merge
merged_df = pd.merge(traffic, PM10, left_on='datetime', right_on='datetime', how='inner')

print(len(merged_df))
merged_df.drop_duplicates(subset=['datetime'], inplace=True)
print(len(merged_df))
# #count nan
# print(merged_df.isna().sum())


30944
8759


In [9]:
print(merged_df.head)

<bound method NDFrame.head of          station_id  quality  num_vehicles  avg_speed  num_cars  \
0      1.001010e+14     0.75          37.0       30.0      28.0   
4      1.001010e+14     1.00          42.0       40.0      33.0   
8      1.001010e+14     1.00          46.0       36.0      42.0   
12     1.001010e+14     1.00          42.0       43.0      38.0   
16     1.001010e+14     1.00          46.0       37.0      40.0   
...             ...      ...           ...        ...       ...   
30934           NaN      NaN           NaN        NaN       NaN   
30936           NaN      NaN           NaN        NaN       NaN   
30938           NaN      NaN           NaN        NaN       NaN   
30940           NaN      NaN           NaN        NaN       NaN   
30942           NaN      NaN           NaN        NaN       NaN   

       avg_car_speed  num_trucks  avg_truck_speed            datetime  \
0               34.0         9.0             18.0 2023-01-05 12:00:00   
4               41.